In [ ]:
from nbdev import *
%nbdev_default_export functions

Cells will be exported to pct.functions,
unless a different module is specified after an export flag: `%nbdev_export special.module`


In [ ]:
#hide
#%load_ext autoreload
#%autoreload 2

In [ ]:
import numpy as np
import logging
log = logging.getLogger(__name__)

# Functions

Functions that form the elements of a perceptual control node (system).

In [ ]:
%nbdev_export
from abc import ABC, abstractmethod

In [ ]:
%nbdev_export
class BaseFunction(ABC):
    "Base class of a PCT function."
    def __init__(self, name):
        self.value = np.zeros(1)
        self.links = []
        self.name = name
        
    @abstractmethod
    def __call__(self, verbose=False):
        if verbose :
            print(f'{self.value}', end= " ")
            
        return self.value
    
    @abstractmethod    
    def summary(self, str):
        print(f'{self.name} {type(self).__name__} ', end = " ")
        if len(str)>0:
            print(f'| {str}', end= " ")
        print(f'| {self.value}', end = " ")
        if len(self.links)>0:
            print(f'| links ', end=" ")
        for link in self.links:
            print(link.get_name(), end= " ")
        print()
        
    def get_name(self):
        return self.name
    
    def set_value(self, value):
        self.value= value
    
    def get_value(self):
        return self.value
    
    def add_link(self, linkfn):
        self.links.append(linkfn)


In [ ]:
%nbdev_export
class Variable(BaseFunction):
    "A function that returns a variable value."
    def __init__(self, variable, name="variable"):
        super().__init__(name)
        self.value = variable
    
    def __call__(self, verbose=False):
        return super().__call__(verbose)
    
    def summary(self):
        super().summary("")


In [ ]:
%nbdev_export
class Constant(BaseFunction):
    "A function that returns a constant value."
    def __init__(self, constant, name="constant"):
        super().__init__(name)
        self.value = constant
    
    def __call__(self, verbose=False):
        return super().__call__(verbose)
    
    def summary(self):
        super().summary("")


In [ ]:
%nbdev_export
class Subtract(BaseFunction):
    "A function that subtracts one value from another."
    def __init__(self, name="subtract"):
        super().__init__(name)
    
    def __call__(self, verbose=False):
        self.value = self.links[0].get_value()-self.links[1].get_value()

        return super().__call__(verbose)

    def summary(self):
        super().summary("")


In [ ]:
%nbdev_export
class Proportional(BaseFunction):
    "Proportional function."
    def __init__(self, gain, name="proportional"):
        super().__init__(name)
        self.gain = gain
    
    def __call__(self, verbose=False):
        self.output = input * self.gain
        return super().__call__(verbose)
    
    def summary(self):
        super().summary(f' gain {self.gain}')


In [ ]:
%nbdev_export
class Integration(BaseFunction):
    "Integration function."
    def __init__(self, gain, slow, name="integration"):
        super().__init__(name)
        self.gain = gain
        self.slow = slow
    
    def __call__(self, verbose=False):
        input = self.links[0].get_value()
        self.value = self.value +  ((input * self.gain) - self.value)/self.slow
        
        return super().__call__(verbose)

 
    def summary(self):
        super().summary(f'gain {self.gain} slow {self.slow} ')



In [ ]:
def velocity_model(velocity,  force , mass):
    velocity = velocity + force / mass
    return velocity

In [ ]:
integrator = Integration(3, 10)
integrator.add_link(Constant(5))
output = integrator()
print(output)

[1.5]


In [ ]:
#hide
integrator.set_value(np.array([0]))
output = integrator()
assert output == [1.5]

In [ ]:
%nbdev_export
class PCTNode():
    "A single PCT controller."
    def __init__(self, perception, name="pctnode"):
        self.links_built = False
        self.name=name 
        self.perceptionCollection = [perception]
        reference = Constant(1)
        self.referenceCollection = [reference]
        comparator = Subtract()
        self.comparatorCollection = [comparator]
        self.outputCollection = [Integration(75, 100)]
    
    def __call__(self, verbose=False):
        if not self.links_built:
            self.build_links()
            
        for referenceFunction in self.referenceCollection:
            referenceFunction(verbose)               

        for perceptionFunction in self.perceptionCollection:
            perceptionFunction(verbose)
                    
        for comparatorFunction in self.comparatorCollection:
            comparatorFunction(verbose)

        for outputFunction in self.outputCollection:
            outputFunction(verbose)
            
        self.output = self.outputCollection[-1].get_value()
        
        if verbose:
            print()
            
        return self.output
    
    def build_links(self):
        if len(self.referenceCollection)>0:
            link = self.referenceCollection[0]
            for i in range (1, len(self.referenceCollection)):
                self.referenceCollection[i].add_link(link)               
                link = self.referenceCollection[i]

        if len(self.perceptionCollection)>0:
            link = self.perceptionCollection[0]
            for i in range (1, len(self.perceptionCollection)):
                self.perceptionCollection[i].add_link(link)               
                link = self.perceptionCollection[i]

        self.comparatorCollection[0].add_link(self.referenceCollection[-1])
        self.comparatorCollection[0].add_link(self.perceptionCollection[-1])

        if len(self.comparatorCollection)>1:
            link = self.comparatorCollection[1]
            for i in range (1, len(self.comparatorCollection)):
                self.comparatorCollection[i].add_link(link)               
                link = self.comparatorCollection[i]

        self.outputCollection[0].add_link(self.comparatorCollection[-1])

        if len(self.outputCollection)>0:
            link = self.outputCollection[0]
            for i in range (1, len(self.outputCollection)):
                self.outputCollection[i].add_link(link)               
                link = self.outputCollection[i]

        self.links_built = True

    def run(self, steps=None, verbose=False):
        for i in range(steps):
            out = self(verbose)
        return out
    
    def set_output(self, value):
        self.outputCollection[-1].set_value(value)
        
    def get_output_function(self):
        return self.outputCollection[-1]
    
    def summary(self):
        if not self.links_built:
            self.build_links()

        print(self.name, type(self).__name__)
        print("----------------------------")
        print("REF:", end=" ")
        for referenceFunction in self.referenceCollection:
            referenceFunction.summary()   
        
        print("PER:", end=" ")
        for perceptionFunction in self.perceptionCollection:
            perceptionFunction.summary()
        
        print("COM:", end=" ")
        for comparatorFunction in self.comparatorCollection:
            comparatorFunction.summary()
        
        print("OUT:", end=" ")
        for outputFunction in self.outputCollection:
            outputFunction.summary()
        
        print("----------------------------")


In [ ]:
mass = 50
force = 0
velocity = Variable(0, name="velocity")
node = PCTNode(velocity)

In [ ]:
node.summary()

pctnode PCTNode
----------------------------
REF: constant Constant  | 1 
PER: velocity Variable  | 0 
COM: subtract Subtract  | [0.] | links  constant velocity 
OUT: integration Integration  | gain 75 slow 100  | [0.] | links  subtract 
----------------------------


In [ ]:
output = node(verbose=True)
force = output[0]
velocity.set_value(velocity_model(velocity.get_value(), force, mass))
print(node.get_output_function().get_value())
print(force)
assert output[0] == 0.75

1 0 1 [0.75] 
[0.75]
0.75


In [ ]:
node.set_output(0)
velocity.set_value(0)
#output = node.run(100, True)
for i in range(100):
    force = node(verbose=True)
    #print(force)
    vel = velocity_model(velocity.get_value(), force, mass)
    #print(vel)
    velocity.set_value(vel)
    
print(force)

1 0 1 0.75 
1 0.015 0.985 1.48125 
1 0.044625 0.955375 2.1829687499999997 
1 0.088284375 0.911715625 2.8449257812499997 
1 0.145182890625 0.854817109375 3.4575893554687496 
1 0.21433467773437498 0.785665322265625 4.012262453613281 
1 0.2945799268066406 0.7054200731933594 4.501204883972168 
1 0.38460402448608394 0.615395975513916 4.917739816767884 
1 0.4829588208214416 0.5170411791785584 5.256343302984123 
1 0.5880856868811241 0.41191431311887594 5.512715604793439 
1 0.6983399989769928 0.30166000102300716 5.68383344951276 
1 0.812016667967248 0.18798333203275197 5.767982614042197 
1 0.927376320248092 0.07262367975190798 5.764770547715705 
1 1.042671731202406 -0.04267173120240608 5.675119043836744 
1 1.156174112079141 -0.1561741120791409 5.50123726933902 
1 1.2661988574659213 -0.2661988574659213 5.246575753546189 
1 1.371130372536845 -0.3711303725368451 4.915762216608093 
1 1.4694456168690069 -0.4694456168690069 4.514520381790257 
1 1.559736024504812 -0.559736024504812 4.049573159593746 

In [ ]:
#hide
from nbdev import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_rmath.ipynb.
Converted 02_functions.ipynb.
Converted 03_nodes.ipynb.
Converted 04_hierarchy.ipynb.
Converted index.ipynb.
Converted Untitled.ipynb.
